In [8]:
import Ipynb_importer
import cv2 as cv
import sys,os
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from numba import jit
from image_preprocessing import cdi_process,calc_entropy
from sklearn.preprocessing import StandardScaler,MinMaxScaler

%matplotlib inline
sns.set_style('darkgrid')

Home_os='C:/Users/jingy/Jupyter/Crime/Fire/Dataset/video/'
negative_os=Home_os+'negative/'
positive_os=Home_os+'positive/'

In [19]:
def calc_prob_entropy(magnitude,angle,bin_num=[4,4],max_num=[20,2*np.pi]):
        hist_2d=np.zeros((bin_num[0],bin_num[1]))
        for m,a in zip(magnitude.ravel(),angle.ravel()):
            x=m//(max_num[0]/bin_num[0])
            y=a//(max_num[1]/bin_num[1])
            
            x= bin_num[0]-1 if x>=bin_num[0]-1 else int(x)
            y= bin_num[1]-1 if y>=bin_num[1]-1 else int(y)
            
            hist_2d[x,y]+=1
            
        prob=hist_2d/(224*224)
        
        H=np.sum([(-i*(np.log2(i))) for i in prob.ravel() if i !=0.0])
        
        return prob,H

In [29]:
video=os.listdir(negative_os)
Intensity_list=[]
for i in video:
    start=0
    video_name=negative_os+i
    cap = cv.VideoCapture(video_name)
    My_CDI=cdi_process.CDI_lib(cap,Threshold=False,mythold=[0,255])
    in_list=[]
    while 1:
        if start<=26:
            cdi,frame=My_CDI.CDI(start,threshold_type=cv.THRESH_TOZERO)
            cdi_next,frame_next=My_CDI.CDI(start+1,threshold_type=cv.THRESH_TOZERO)
            cdi=cv.normalize(cdi,None,0,255,cv.NORM_MINMAX)
            cdi_next=cv.normalize(cdi_next,None,0,255,cv.NORM_MINMAX)
#             hsv = np.zeros_like(frame)
#             hsv[...,1] = 255
            flow = cv.calcOpticalFlowFarneback(cdi,cdi_next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag=np.sqrt(flow[...,0]**2+flow[...,1]**2)
            ang=np.arctan2(flow[...,1],flow[...,0])% (2 * np.pi)
            ang=np.degrees(ang)
            prob = cv.calcHist([ang], [0], None, [361], [0, 360])/(224*224)
            H=np.sum([(-i*(np.log2(i))) for i in prob.ravel() if i !=0.0])
#             hsv[...,0] = ang*180/np.pi/2  #角度
#             hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX) 
#             bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
            #H=calc_entropy.entropy(hsv[...,2])
            #_,H=calc_prob_entropy(magnitude=mag,angle=ang)
            in_list.append(H)
            cv.imshow('frame',frame)
            cv.imshow('cdi',cdi)
#             cv.imshow('optical',bgr)
            k = cv.waitKey(1) 
            start+=1
            if k ==27:     
                break 
        else:
            break
    Intensity_list.append(in_list)
    cap.release()
cv.destroyAllWindows() 

In [30]:
video=os.listdir(positive_os)
P_Intensity_list=[]
for i in video:
    start=0
    video_name=positive_os+i
    cap = cv.VideoCapture(video_name)
    My_CDI=cdi_process.CDI_lib(cap,Threshold=False,mythold=[0,255])
    in_list=[]
    while 1:
        if start<=26:
            cdi,frame=My_CDI.CDI(start,threshold_type=cv.THRESH_TOZERO)
            cdi_next,frame_next=My_CDI.CDI(start+1,threshold_type=cv.THRESH_TOZERO)
            cdi=cv.normalize(cdi,None,0,255,cv.NORM_MINMAX)
            cdi_next=cv.normalize(cdi_next,None,0,255,cv.NORM_MINMAX)
#             hsv = np.zeros_like(frame)
#             hsv[...,1] = 255
            flow = cv.calcOpticalFlowFarneback(cdi,cdi_next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag=np.sqrt(flow[...,0]**2+flow[...,1]**2)
            ang=np.arctan2(flow[...,1],flow[...,0])% (2 * np.pi)
            ang=np.degrees(ang)
            prob = cv.calcHist([ang], [0], None, [361], [0, 360])/(224*224)
            H=np.sum([(-i*(np.log2(i))) for i in prob.ravel() if i !=0.0])
#             hsv[...,0] = ang*180/np.pi/2  #角度
#             hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX) 
#             bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
            #H=calc_entropy.entropy(hsv[...,2])
            #_,H=calc_prob_entropy(magnitude=mag,angle=ang)
            in_list.append(H)
            cv.imshow('frame',frame)
            cv.imshow('cdi',cdi)
#             cv.imshow('optical',bgr)
            k = cv.waitKey(1) 
            start+=1
            if k ==27:     
                break 
        else:
            break
    P_Intensity_list.append(in_list)
    cap.release()
cv.destroyAllWindows() 

In [31]:

Combine_Entropy=Intensity_list+P_Intensity_list
# min_max_scaler =MinMaxScaler()
# for i in range(len(Combine_Entropy)):
#     Combine_Entropy[i] = ((min_max_scaler.fit_transform(np.array(Combine_Entropy[i]).reshape(-1,1)))).reshape(-1).tolist()
new_list=[]
for i in Combine_Entropy:
        str1 = ' '.join([str(x) for x in i])
        new_list.append(str1)

print(len(new_list))


268


In [32]:
x = [0] * 126
y =[1]*142
z=x+y

In [33]:
#combine_dict={'entropy':new_list,'label':z}
combine_dict1={'entropy':new_list,'label':z}
df=pd.DataFrame(combine_dict1)

display(df)

y=df['label'].values

,entropy,label
0,8.338194 8.302568 8.313569 8.318121 8.034195 8...,0
1,8.279738 8.239868 8.173103 8.180133 8.131359 8...,0
2,8.442705 8.459726 8.451424 8.45746 8.469121 8....,0
3,8.33338 8.355842 8.376957 8.300938 8.3071575 8...,0
4,8.304228 8.314566 8.358588 8.370737 8.408796 8...,0
...,...,...
263,8.264909 8.287047 8.269607 8.186451 8.211284 8...,1
264,7.8903666 7.9138727 7.47139 7.7778797 8.172076...,1
265,4.826669 4.6771564 8.186701 8.0379305 8.176149...,1
266,8.423625 8.33579 8.37011 7.6386585 8.289253 8....,1


In [1]:
data_df = df['entropy'].str.split(' ', expand=True)

data_df['Class'] =y
data_df.to_csv('C:/Users/jingy/Jupyter/Crime/Fire/CDI_Based/Entropy/CSV/Optical_Ang.csv')
display(data_df)

NameError: name 'df' is not defined